In [1]:
import pandas as pd
import glob, os
import re
import fnmatch

## This will prepare the FDIC data for the Redshift database (even though it says that it does it for the NCUA data. 
    1) Goes to the directory where all the files are stored
        a. Searches through all the files that contain fs220
        b. Append the endings of these files to a list [fs220A, fs220B, fs220C, etc...]
        c. Get the unique endings from that list by taking the set()
        d. Now we have all the unique endings (we will make a seperate table in our Redshift Database for each)
        
    2) For each ending, we find the most recent year and month of the reported data 
       This is needed to get the latest_vars variable
        a. First it finds most recent year reported for that ending (for loop with > comparisons)
        b. Then it finds most recent month reported in that year for that ending (for loop with > comparisons)
        c. It appends each year and month combination for each ending to lists
          (*NOTE: this is necessary as there is not a universal latest year and month for each ending. 
          For example, fs220E does not report in 201706, but instead, its last reporting is in 201606)
          
    3) Using the zip() function, we can iterate over three lists at a time (ending list, latest year list, latest month list) 
         a. We get the latest vars for that ending
         b. We then run through the main loop with that ending and complete the steps as before:
            i.    Reads in all fs220(ending) files 
            ii.   Makes all the column names uppercase
            iii.  Compares columns names to columns names of the latest variables using list comprehension
            iv.   If df is missing latest columns, includes them and fills with NaN, this is to maintain order - COPY into 
                  Redshift
            v.    Adds an ORDERS column which is a sequential index
            vi.   Adds a unique identifier with CU_NUM_DATE 
            vii.  Adds two columns, QUARTER, and YEAR, in case we may use these to query later on
            viii. Final structure is all df's(all years, months, and endings) with all the latest columns in a dictionary 
                  The df's are all "standardized"
                  If latest column doesn't exist in the original df, it is filled with NaN's. 
                  (*NOTE: we made need to change what it is filled in with here since NaN is a float and we really want   
                  NULL/None for Redshift Database to read correctly)
    
    4) Write all the dictionary files to a new folder to load into AWS S3 bucket 
       Add naming convention of same prefix for each file.
       Example: fs220A-csv. is prefix for all fs220A files (all years and months)

### Code

In [2]:
# Get columns names for all reports in the data lake.
path='C:/Analytics/DATA911/Arkatechture/FDIC_Data/FDIC_Lake/'
os.chdir(path)
filelist=[]
dict_={}
startvalue=1
prefix='All_Reports_'
day_dict={'03':'31','06':'30','09':'30','12':'31'}

unique_endings=[]
# All_Reports_ is the prefix for the FDIC files. Get list of all that are in
# the FDIC data lake. Then get the unique endings -- i.e., filenames.
for file in glob.glob("All_Reports_*"):
    unique_endings.append(file.split("_")[-1].split(".")[0])    
unique_endings=set(unique_endings)

# Find the last year and date reported.  Note that the first part of
# the unique file endings is the date in the form yyyymmdd.
last_year_list=[]
last_month_list=[]
for ending in unique_endings:
    last_year_reported='2000'
    last_month_reported=3
    for file in glob.glob("*"+ending+'.csv'):
        latest_year=file[12:16]
        latest_year_int=int(latest_year)
        if latest_year_int > int(last_year_reported):
            last_year_reported = str(latest_year_int)
    last_year_list.append(last_year_reported)
    for file in glob.glob("*"+last_year_reported+"*"+ending+'.csv'):
        latest_month=file[16:18]
        len_s=len(latest_month)
        latest_month_int=int(latest_month)
        if latest_month_int > int(last_month_reported):
                last_month_reported = str(latest_month_int).rjust(len_s, "0")
    last_month_list.append(last_month_reported)


for ending,year,month in zip(unique_endings, last_year_list, last_month_list):
    #print(year,month,ending)
    latest_csv=pd.read_csv(path+prefix+year+month+day_dict[month]+'_'+ending+'.csv', encoding='latin1', dtype=object)#/most_recent+ending
    latest_csv.columns=map(str.upper, latest_csv.columns)
    latest_vars=latest_csv.columns.tolist()

    for file in glob.glob("*_"+ending+".csv"):#200903_fs220.csv
        filename = file[:-4]
        filelist.append(filename)
        dict_[filename] = pd.read_csv(file, encoding='latin1', dtype=object)
        dict_[filename].columns = map(str.upper, dict_[filename].columns)
        set1=dict_[filename].columns.tolist()
        set2=latest_vars
        to_add=[x for x in set2 if x not in set1]
        columns=dict_[filename].columns.tolist()
        dict_[filename]=dict_[filename].reindex(columns=[*dict_[filename].columns.tolist(), *to_add], fill_value='nan')
        dict_[filename]=dict_[filename].reindex_axis(columns+to_add, axis=1)
        dict_[filename].insert(0, 'ORDER', range(startvalue, startvalue + len(dict_[filename])))
        startvalue+=len(dict_[filename])
        dict_[filename].insert(1, 'FED_RSSD_REPDTE', dict_[filename]['FED_RSSD'].astype(str)+
                      '_'+pd.to_datetime(dict_[filename]['REPDTE']).dt.date.astype(str))
        dict_[filename].insert(2, 'QUARTER', filename[16:18])
        dict_[filename].insert(3, 'YEAR', filename[12:16])

### Code turned into funtion -- this code was not modified for FDIC.
    Same as above, just a function. All you have to do is input the 'path' where all your files are located. 
    Dict_ is returned. As seen here, we resave it as the variable dict_.

In [ ]:
def prepare_files(path):
    path=path
    os.chdir(path)
    filelist=[]
    dict_={}
    startvalue=1

    unique_endings=[]
    for file in glob.glob("*_fs220*"):
        unique_endings.append(file.split("_")[-1].split(".")[0])
    unique_endings=set(unique_endings)


    last_year_list=[]
    last_month_list=[]
    for ending in unique_endings:
        last_year_reported='2000'
        last_month_reported=3
        for file in glob.glob("*"+ending+'.csv'):
            latest_year=file[0:4]
            latest_year_int=int(latest_year)
            if latest_year_int > int(last_year_reported):
                last_year_reported = str(latest_year_int)
        last_year_list.append(last_year_reported)
        for file in glob.glob(last_year_reported+"*"+ending+'.csv'):
                latest_month=file[4:6]
                len_s=len(latest_month)
                latest_month_int=int(latest_month)
                if latest_month_int > int(last_month_reported):
                    last_month_reported = str(latest_month_int).rjust(len_s, "0")
                    last_month_list.append(last_month_reported)


    for ending,year,month in zip(unique_endings, last_year_list, last_month_list):
        #print(year,month,ending)
        latest_csv=pd.read_csv(path+year+month+'_'+ending+'.csv', encoding='latin1', dtype=object)#/most_recent+ending
        latest_csv.columns=map(str.upper, latest_csv.columns)
        latest_vars=latest_csv.columns.tolist()

        for file in glob.glob("*_"+ending+".csv"):#200903_fs220.csv
            filename = file[:-4]
            filelist.append(filename)
            dict_[filename] = pd.read_csv(file, encoding='latin1', dtype=object)
            dict_[filename].columns = map(str.upper, dict_[filename].columns)
            set1=dict_[filename].columns.tolist()
            set2=latest_vars
            to_add=[x for x in set2 if x not in set1]
            columns=dict_[filename].columns.tolist()
            dict_[filename]=dict_[filename].reindex(columns=[*dict_[filename].columns.tolist(), *to_add], fill_value='nan')
            dict_[filename]=dict_[filename].reindex_axis(columns+to_add, axis=1)
            dict_[filename].insert(0, 'ORDERS', range(startvalue, startvalue + len(dict_[filename])))
            startvalue+=len(dict_[filename])
            dict_[filename].insert(1, 'CU_NUM_DATE', dict_[filename]['CU_NUMBER'].astype(str)+'_'+pd.to_datetime(dict_[filename]['CYCLE_DATE']).dt.date.astype(str))
            dict_[filename].insert(2, 'QUARTER', filename[4:6])
            dict_[filename].insert(3, 'YEAR', filename[:-8])
    return dict_

In [ ]:
# Function to do upper cells.  (DO NOT RUN)
dict_=prepare_files('C:/Users/Owner/Documents/Practicum Projects/Arkatechture/NCUA_csv_2009_2017/')

In [66]:
dict_['All_Reports_20100331_Cash Dividends']

,ORDER,FED_RSSD_REPDTE,QUARTER,YEAR,CERT,DOCKET,FED_RSSD,RSSDHCR,NAME,CITY,...,FLDOFF,FED,OCCDIST,OTSREGNM,OFFOA,CB,INST.WEBADDR,EQCDIV,EQCDIVP,EQCDIVC
0,1,3317192_2010-03-31,03,2010,57899,0,3317192,3370517,1st Advantage Bank,Saint Peters,...,St. Louis,St. louis,Western,West,0,1,www.1stadvantagebank.com,0,0,0
1,2,564856_2010-03-31,03,2010,22039,0,564856,1128415,1st Bank,Sidney,...,Billings,Minneapolis,Western,West,0,1,www.our1stbank.com,40,0,40
2,3,888253_2010-03-31,03,2010,16522,0,888253,2003975,1ST BANK,Evanston,...,Salt Lake City,Kansas City,Western,West,0,0,NaN,0,0,0
3,4,419255_2010-03-31,03,2010,16419,17301,419255,1055780,1st Bank & Trust,Broken Bow,...,Tulsa,Kansas City,Southern,West,0,1,http://www/1stbankandtrust.com,350,0,350
4,5,148470_2010-03-31,03,2010,30367,5236,148470,0,1st Bank of Sea Isle City,Sea Isle City,...,Jamesburg,Philadelphia,Northeastern,Northeast,0,1,www.1stbankseaisle.com,0,0,0
5,6,488055_2010-03-31,03,2010,4788,0,488055,1055904,1st Bank of Troy,Troy,...,Kansas City,Kansas City,Western,Western,0,1,NaN,35,0,35
6,7,589671_2010-03-31,03,2010,31826,7509,589671,1058165,1st Bank Oklahoma,Claremore,...,Tulsa,Kansas City,Southern,West,0,1,NaN,0,0,0
7,8,3048487_2010-03-31,03,2010,57298,0,3048487,3559844,1st Bank Yuma,Yuma,...,Phoenix,San Francisco,Western,Western,0,1,http://www.1stbankyuma.com,0,0,0
8,9,941653_2010-03-31,03,2010,32487,8322,941653,1064791,1st Cameron State Bank,Cameron,...,Kansas City,Kansas City,Western,West,0,1,www.1stcameron.com,0,0,0
9,10,3594797_2010-03-31,03,2010,58485,0,3594797,0,1st Capital Bank,Monterey,...,San Francisco,San Francisco,Western,Western,0,1,www.1stcapital.bank,0,0,0


### Final Step: Write all standardized files to new folder for S3 load

In [3]:
# file_path='C:/Analytics/DATA911/Arkatechture/FDIC_Data/S3/'
# directory = os.path.dirname(file_path)
for key in dict_.keys():
    dict_[key].to_csv('C:/Analytics/DATA911/Arkatechture/FDIC_Data/S3/{}.csv'.format(key.split("_")[-1]+'-csv.'+key), index=False)